# Libraries

## Install

In [1]:
!pip install werkzeug
!pip install bert-extractive-summarizer

In [2]:
!pip install jupyter-client==6.1.5
!pip install wrapt==1.12.1
!pip install traitlets==5.0
!pip install typed-ast
!pip install pytest-filter-subpackage
!pip install pytest-cov
!pip install tensorflow --user
!pip install pyvi

In [3]:
!pip install -r requirements.txt

  Using cached transformers-3.3.0-py3-none-any.whl (1.1 MB)
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.61.2
    Uninstalling tqdm-4.61.2:
      Successfully uninstalled tqdm-4.61.2
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.1
    Uninstalling srsly-2.4.1:
      Successfully uninstalled srsly-2.4.1
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.5
    Uninstalling preshed-3.0.5:
      Successfully uninstalled preshed-3.0.5
  Attempting uninstall: blis
    Found existing installation: blis 0.7.4
    Uninstalling blis-0.7.4:
      Successfully uninstalled blis-0.7.4
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.8
    Uninstalling thinc-8.0.8:
      Successfully uninstalled thinc-8.0.8
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 3.2.0
    Uninstalling jsonschema-3.2.0:
      Successfully uni

In [4]:
!pip install neuralcoref --no-binary neuralcoref

In [2]:
!pip install rouge-metric

In [ ]:
!pip install pandas

## Import

In [1]:
from typing import List, Optional, Tuple, Union
from typing import Dict
from transformers import (PreTrainedModel, PreTrainedTokenizer,
                          BertModel, BertTokenizer,
                          GPT2Model, GPT2Tokenizer,
                          BartModel, BartTokenizer, 
                          OpenAIGPTModel, OpenAIGPTTokenizer, 
                          CTRLModel, CTRLTokenizer, 
                          TransfoXLModel, TransfoXLTokenizer, 
                          XLNetModel, XLNetTokenizer,
                          XLMModel, XLMTokenizer, 
                          DistilBertModel, DistilBertTokenizer,
                          AlbertModel, AlbertTokenizer,
                          AutoModel, AutoTokenizer)

from transformers import *
import numpy as np
import torch
import neuralcoref
import spacy

from numpy import ndarray
from spacy.lang.vi import Vietnamese

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

In [2]:
from rouge_metric import PyRouge

In [3]:
import pandas as pd

# Setup classes & functions

In [4]:
import logging
logging.basicConfig(level=logging.WARNING)
class BertParent(object):
    """
    Base handler for BERT models.
    """
    MODELS = {
                'bert-base-multilingual-uncased': (BertModel, BertTokenizer),
                'bert-base-uncased': (BertModel, BertTokenizer),
                'bert-large-uncased': (BertModel, BertTokenizer),
                'gpt2': (GPT2Model, GPT2Tokenizer),
                'facebook/bart-large': (BartModel, BartTokenizer),
                'openai-gpt': (OpenAIGPTModel, OpenAIGPTTokenizer),
                'ctrl': (CTRLModel, CTRLTokenizer),
                'transfo-xl-wt103': (TransfoXLModel, TransfoXLTokenizer), 
                'xlnet-large-cased': (XLNetModel, XLNetTokenizer),
                'xlnet-base-cased': (XLNetModel, XLNetTokenizer),
                'xlm-mlm-enfr-1024': (XLMModel, XLMTokenizer),
                'distilbert-base-uncased': (DistilBertModel, DistilBertTokenizer),
                'albert-base-v2': (AlbertModel, AlbertTokenizer),
                'albert-large-v2': (AlbertModel, AlbertTokenizer),
                'allenai/scibert_scivocab_uncased': (AutoModel, AutoTokenizer),
                'vinai/phobert-large': (AutoModel, AutoTokenizer)
    }

    def __init__(self,
                model: str,
                custom_model: PreTrainedModel = None,
                custom_tokenizer: PreTrainedTokenizer = None):
        """
        :param model: Model is the string path for the bert weights. If given a keyword, the s3 path will be used.
        :param custom_model: This is optional if a custom bert model is used.
        :param custom_tokenizer: Place to use custom tokenizer.
        """
        base_model, base_tokenizer = self.MODELS.get(model, (None, None))

        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

        if custom_model:
            self.model = custom_model.to(self.device)
        else:
            self.model = base_model.from_pretrained(model, output_hidden_states=True).to(self.device)

        if custom_tokenizer:
            self.tokenizer = custom_tokenizer
        else:
            self.tokenizer = base_tokenizer.from_pretrained(model)

        self.model.eval()

    def tokenize_input(self, text: str) -> torch.tensor:
        """
        Tokenizes the text input.
        :param text: Text to tokenize.
        :return: Returns a torch tensor.
        """
        tokenized_text = self.tokenizer.tokenize(text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        return torch.tensor([indexed_tokens]).to(self.device) 

    def extract_embeddings(self,
                        text: str,
                        hidden: Union[List[int], int] = -2,
                        reduce_option: str = 'mean',
                        hidden_concat: bool = False) -> torch.Tensor:
        """
        Extracts the embeddings for the given text.
        :param text: The text to extract embeddings for.
        :param hidden: The hidden layer(s) to use for a readout handler.
        :param squeeze: If we should squeeze the outputs (required for some layers).
        :param reduce_option: How we should reduce the items.
        :param hidden_concat: Whether or not to concat multiple hidden layers.
        :return: A torch vector.
        """
        tokens_tensor = self.tokenize_input(text)
        pooled, hidden_states = self.model(tokens_tensor)[-2:]

        # deprecated temporary keyword functions.
        
        if reduce_option == 'concat_last_4':
            last_4 = [hidden_states[i] for i in (-1, -2, -3, -4)]
            cat_hidden_states = torch.cat(tuple(last_4), dim=-1)
            return torch.mean(cat_hidden_states, dim=1).squeeze()

        elif reduce_option == 'reduce_last_4':
            last_4 = [hidden_states[i] for i in (-1, -2, -3, -4)]
            return torch.cat(tuple(last_4), dim=1).mean(axis=1).squeeze()

        elif type(hidden) == int:
            hidden_s = hidden_states[hidden]
            return self._pooled_handler(hidden_s, reduce_option)

        elif hidden_concat:
            last_states = [hidden_states[i] for i in hidden]
            cat_hidden_states = torch.cat(tuple(last_states), dim=-1)
            return torch.mean(cat_hidden_states, dim=1).squeeze()

        last_states = [hidden_states[i] for i in hidden]
        hidden_s = torch.cat(tuple(last_states), dim=1)

        return self._pooled_handler(hidden_s, reduce_option)
    
    def create_matrix(self,
                    content: List[str],
                    hidden: Union[List[int], int] = -2,
                    reduce_option: str = 'mean',
                    hidden_concat: bool = False) -> ndarray:
        """
        Create matrix from the embeddings.
        :param content: The list of sentences.
        :param hidden: Which hidden layer to use.
        :param reduce_option: The reduce option to run.
        :param hidden_concat: Whether or not to concat multiple hidden layers.
        :return: A numpy array matrix of the given content.
        """
        
        return np.asarray([
            np.squeeze(self.extract_embeddings(t, hidden=hidden, reduce_option=reduce_option, 
                                               hidden_concat=hidden_concat).data.cpu().numpy()) for t in content])
    
    def _pooled_handler(self, hidden: torch.Tensor,
                        reduce_option: str) -> torch.Tensor:
        """
        Handles torch tensor.

        :param hidden: The hidden torch tensor to process.
        :param reduce_option: The reduce option to use, such as mean, etc.
        :return: Returns a torch tensor.
        """

        if reduce_option == 'max':
            return hidden.max(dim=1)[0].squeeze()

        elif reduce_option == 'median':
            return hidden.median(dim=1)[0].squeeze()

        return hidden.mean(dim=1).squeeze()

    def __call__(self,
                content: List[str],
                hidden: int = -2,
                reduce_option: str = 'mean',
                hidden_concat: bool = False) -> ndarray:
        """
        Create matrix from the embeddings.

        :param content: The list of sentences.
        :param hidden: Which hidden layer to use.
        :param reduce_option: The reduce option to run.
        :param hidden_concat: Whether or not to concat multiple hidden layers.
        :return: A numpy array matrix of the given content.
        """
        return self.create_matrix(content, hidden, reduce_option, hidden_concat)

In [5]:
class SentenceHandler(object):
    def __init__(self, language=Vietnamese):
        """
        Base Sentence Handler with Spacy support.
        :param language: Determines the language to use with spacy.
        """
        self.nlp = language()
        try:
            # Supports spacy 2.0
            self.nlp.add_pipe(self.nlp.create_pipe('sentencizer'))
            self.is_spacy_3 = False
        except Exception:
            # Supports spacy 3.0
            self.nlp.add_pipe("sentencizer")
            self.is_spacy_3 = True

    def sentence_processor(self, doc,
                           min_length: int = 40,
                           max_length: int = 600) -> List[str]:
        """
        Processes a given spacy document and turns them into sentences.
        :param doc: The document to use from spacy.
        :param min_length: The minimum length a sentence should be to be considered.
        :param max_length: The maximum length a sentence should be to be considered.
        :return: Sentences.
        """
        to_return = []

        for c in doc.sents:
            if max_length > len(c.text.strip()) > min_length:

                if self.is_spacy_3:
                    to_return.append(c.text.strip())
                else:
                    to_return.append(c.string.strip())

        return to_return

    def process(self, body: str,
                min_length: int = 40,
                max_length: int = 600) -> List[str]:
        """
        Processes the content sentences.

        :param body: The raw string body to process
        :param min_length: Minimum length that the sentences must be
        :param max_length: Max length that the sentences mus fall under
        :return: Returns a list of sentences.
        """
        doc = self.nlp(body)
        return self.sentence_processor(doc, min_length, max_length)

    def __call__(self, body: str,
                 min_length: int = 40,
                 max_length: int = 600) -> List[str]:
        """
        Processes the content sentences.

        :param body: The raw string body to process
        :param min_length: Minimum length that the sentences must be
        :param max_length: Max length that the sentences mus fall under
        :return: Returns a list of sentences.
        """
        return self.process(body, min_length, max_length)
# removed previous import and related functionality since it's just a blank language model,
# while neuralcoref requires passing pretrained language model via spacy.load() or spacy.lang.vi
class CoreferenceHandler(SentenceHandler):
    def __init__(self, spacy_model = Vietnamese,
                 greedyness: float = 0.45):
        """
        Corefence handler. Only works with spacy < 3.0.
        :param spacy_model: The spacy model to use as default.
        :param greedyness: The greedyness factor.
        """
        self.nlp = spacy_model()
            
        self.nlp.add_pipe(self.nlp.create_pipe('sentencizer'))
        neuralcoref.add_to_pipe(self.nlp, greedyness=greedyness)

    def process(self, body: str, min_length: int = 40, max_length: int = 600):
        """
        Processes the content sentences.
        :param body: The raw string body to process
        :param min_length: Minimum length that the sentences must be
        :param max_length: Max length that the sentences mus fall under
        :return: Returns a list of sentences.
        """
        doc = self.nlp(body)._.coref_resolved
        doc = self.nlp(doc)
        return [c.string.strip()
                for c in doc.sents
                if max_length > len(c.string.strip()) > min_length]

In [6]:
class ClusterFeatures(object):
    """
    Basic handling of clustering features.
    """
    def __init__(self,
                features: ndarray,
                algorithm: str = 'kmeans',
                pca_k: int = None,
                random_state: int = 12345):
        """
        :param features: the embedding matrix created by bert parent.
        :param algorithm: Which clustering algorithm to use.
        :param pca_k: If you want the features to be ran through pca, this is the components number.
        :param random_state: Random state.
        """
        if pca_k: self.features = PCA(n_components=pca_k).fit_transform(features)
        else: self.features = features

        self.algorithm = algorithm
        self.pca_k = pca_k
        self.random_state = random_state

    def __get_model(self, k: int):
        """
        Retrieve clustering model.
        :param k: amount of clusters.
        :return: Clustering model.
        """
        if self.algorithm == 'gmm':
            return GaussianMixture(n_components=k, random_state=self.random_state)
        return KMeans(n_clusters=k, random_state=self.random_state)

    def __get_centroids(self, model):
        """
        Retrieve centroids of model.
        :param model: Clustering model.
        :return: Centroids.
        """
        if self.algorithm == 'gmm':
            return model.means_
        return model.cluster_centers_

    def __find_closest_args(self, centroids: np.ndarray) -> Dict:
        """
        Find the closest arguments to centroid.
        :param centroids: Centroids to find closest.
        :return: Closest arguments.
        """
        centroid_min = 1e10
        cur_arg = -1
        args = {}
        used_idx = []

        for j, centroid in enumerate(centroids):
            for i, feature in enumerate(self.features):
                value = np.linalg.norm(feature - centroid)
                if value < centroid_min and i not in used_idx:
                    cur_arg = i
                    centroid_min = value
            used_idx.append(cur_arg)
            args[j] = cur_arg
            centroid_min = 1e10
            cur_arg = -1
        return args

    def cluster(self, ratio: float = 0.1, num_sentences: int = None) -> List[int]:
        """
        Clusters sentences based on the ratio.
        :param ratio: Ratio to use for clustering.
        :param num_sentences: Number of sentences. Overrides ratio.
        :return: Sentences index that qualify for summary.
        """
        if num_sentences is not None:
            if num_sentences == 0: return []
            k = min(num_sentences, len(self.features))
        else:
            k = max(int(len(self.features) * ratio), 1)
        
        #k = max(int(len(self.features) * ratio), 1)
        model = self.__get_model(k).fit(self.features)

        centroids = self.__get_centroids(model)
        cluster_args = self.__find_closest_args(centroids)

        sorted_values = sorted(cluster_args.values())
        return sorted_values

    def calculate_elbow(self, k_max: int) -> List[float]:
        """
        Calculates elbow up to the provided k_max.
        :param k_max: K_max to calculate elbow for.
        :return: The inertias up to k_max.
        """
        inertias = []
        for k in range(1, min(k_max, len(self.features))):
            model = self.__get_model(k).fit(self.features)
            inertias.append(model.inertia_)
        return inertias

    def calculate_optimal_cluster(self, k_max: int):
        """
        Calculates the optimal cluster based on Elbow.
        :param k_max: The max k to search elbow for.
        :return: The optimal cluster size.
        """
        delta_1 = []
        delta_2 = []

        max_strength = 0
        k = 1

        inertias = self.calculate_elbow(k_max)

        for i in range(len(inertias)):
            delta_1.append(inertias[i] - inertias[i-1] if i > 0 else 0.0)
            delta_2.append(delta_1[i] - delta_1[i-1] if i > 1 else 0.0)

        for j in range(len(inertias)):
            strength = 0 if j <= 1 or j == len(
                inertias) - 1 else delta_2[j+1] - delta_1[j+1]

            if strength > max_strength:
                max_strength = strength
                k = j + 1

        return k

    def __call__(self, ratio: float = 0.1, num_sentences: int = None) -> List[int]:
        """
        Clusters sentences based on the ratio.
        :param ratio: Ratio to use for clustering.
        :param num_sentences: Number of sentences. Overrides ratio.
        :return: Sentences index that qualify for summary.
        """
        return self.cluster(ratio, num_sentences)

In [7]:
class ModelProcessor(object):
    aggregate_map = {'mean': np.mean,
                    'min': np.min,
                    'median': np.median,
                    'max': np.max}

    def __init__(self,
                model: str = 'bert-base-multilingual-uncased',
                custom_model: PreTrainedModel = None,
                custom_tokenizer: PreTrainedTokenizer = None,
                hidden: Union[List[int], int] = -2,
                reduce_option: str = 'mean',
                sentence_handler: SentenceHandler = SentenceHandler(),
                random_state: int = 12345,
                hidden_concat: bool = False):
        """
        This is the parent Bert Summarizer model. New methods should implement this class.
        :param model: This parameter is associated with the inherit string parameters from the transformers library.
        :param custom_model: If you have a pre-trained model, you can add the model class here.
        :param custom_tokenizer: If you have a custom tokenizer, you can add the tokenizer here.
        :param hidden: This signifies which layer(s) of the BERT model you would like to use as embeddings.
        :param reduce_option: Given the output of the bert model, this param determines how you want to reduce results.
        :param sentence_handler: The handler to process sentences. If want to use coreference, instantiate and pass.
        CoreferenceHandler instance
        :param random_state: The random state to reproduce summarizations.
        :param hidden_concat: Whether or not to concat multiple hidden layers.
        """
        np.random.seed(random_state)
        self.model = BertParent(model, custom_model, custom_tokenizer)
        self.hidden = hidden
        self.reduce_option = reduce_option
        self.sentence_handler = sentence_handler
        self.random_state = random_state
        self.hidden_concat = hidden_concat

    def cluster_runner(self,
        content: List[str],
        ratio: float = 0.2,
        algorithm: str = 'kmeans',
        use_first: bool = True,
        num_sentences: int = None) -> Tuple[List[str], np.ndarray]:
        """
        Runs the cluster algorithm based on the hidden state. Returns both the embeddings and sentences.

        :param content: Content list of sentences.
        :param ratio: The ratio to use for clustering.
        :param algorithm: Type of algorithm to use for clustering.
        :param use_first: Return the first sentence in the output (helpful for news stories, etc).
        :param num_sentences: Number of sentences to use for summarization.
        :return: A tuple of summarized sentences and embeddings
        """
        
        if num_sentences is not None:
            num_sentences = num_sentences if use_first else num_sentences

        hidden = self.model(
            content, self.hidden, self.reduce_option, hidden_concat=self.hidden_concat)
        hidden_args = ClusterFeatures(
            hidden, algorithm, random_state=self.random_state).cluster(ratio, num_sentences)
        
        if use_first:
            if not hidden_args:
                hidden_args.append(0)

            elif hidden_args[0] != 0:
                hidden_args.insert(0, 0)

        sentences = [content[j] for j in hidden_args]
        embeddings = np.asarray([hidden[j] for j in hidden_args])

        return sentences, embeddings

    def __run_clusters(self,
                    content: List[str],
                    ratio: float = 0.2,
                    algorithm: str = 'kmeans',
                    use_first: bool = True,
                    num_sentences: int = None) -> List[str]:
        """
        Runs clusters and returns sentences.
        :param content: The content of sentences.
        :param ratio: Ratio to use for for clustering.
        :param algorithm: Algorithm selection for clustering.
        :param use_first: Whether to use first sentence
        :param num_sentences: Number of sentences. Overrides ratio.
        :return: summarized sentences
        """
        sentences, _ = self.cluster_runner(
            content, ratio, algorithm, use_first, num_sentences)
        return sentences

    def __retrieve_summarized_embeddings(self,
                                        content: List[str],
                                        ratio: float = 0.2,
                                        algorithm: str = 'kmeans',
                                        use_first: bool = True,
                                        num_sentences: int = None) -> np.ndarray:
        """
        Retrieves embeddings of the summarized sentences.
        :param content: The content of sentences.
        :param ratio: Ratio to use for for clustering.
        :param algorithm: Algorithm selection for clustering.
        :param use_first: Whether to use first sentence
        :return: Summarized embeddings
        """
        _, embeddings = self.cluster_runner(
            content, ratio, algorithm, use_first, num_sentences)
        return embeddings

    def calculate_elbow(self,
                        body: str,
                        algorithm: str = 'kmeans',
                        min_length: int = 40,
                        max_length: int = 600,
                        k_max: int = None) -> List[float]:
        """
        Calculates elbow across the clusters.

        :param body: The input body to summarize.
        :param algorithm: The algorithm to use for clustering.
        :param min_length: The min length to use.
        :param max_length: The max length to use.
        :param k_max: The maximum number of clusters to search.
        :return: List of elbow inertia values.
        """
        sentences = self.sentence_handler(body, min_length, max_length)

        if k_max is None:
            k_max = len(sentences) - 1

        hidden = self.model(sentences, self.hidden,
                            self.reduce_option, hidden_concat=self.hidden_concat)
        elbow = ClusterFeatures(
            hidden, algorithm, random_state=self.random_state).calculate_elbow(k_max)

        return elbow

    def calculate_optimal_k(self,
                            body: str,
                            algorithm: str = 'kmeans',
                            min_length: int = 40,
                            max_length: int = 600,
                            k_max: int = None):
        """
        Calculates the optimal Elbow K.

        :param body: The input body to summarize.
        :param algorithm: The algorithm to use for clustering.
        :param min_length: The min length to use.
        :param max_length: The max length to use.
        :param k_max: The maximum number of clusters to search.
        :return:
        """
        sentences = self.sentence_handler(body, min_length, max_length)

        if k_max is None:
            k_max = len(sentences) - 1

        hidden = self.model(sentences, self.hidden,
                            self.reduce_option, hidden_concat=self.hidden_concat)
        optimal_k = ClusterFeatures(
            hidden, algorithm, random_state=self.random_state).calculate_optimal_cluster(k_max)

        return optimal_k

    def run_embeddings(self,
                    body: str,
                    ratio: float = 0.2,
                    min_length: int = 40,
                    max_length: int = 600,
                    use_first: bool = True,
                    algorithm: str = 'kmeans',
                    num_sentences: int = None,
                    aggregate: str = None) -> Optional[np.ndarray]:
        """
        Preprocesses the sentences, runs the clusters to find the centroids, then combines the embeddings.
        :param body: The raw string body to process
        :param ratio: Ratio of sentences to use
        :param min_length: Minimum length of sentence candidates to utilize for the summary.
        :param max_length: Maximum length of sentence candidates to utilize for the summary
        :param use_first: Whether or not to use the first sentence
        :param algorithm: Which clustering algorithm to use. (kmeans, gmm)
        :param num_sentences: Number of sentences to use. Overrides ratio.
        :param aggregate: One of mean, median, max, min. Applied on zero axis
        :return: A summary embedding
        """
        sentences = self.sentence_handler(body, min_length, max_length)

        if sentences:
            embeddings = self.__retrieve_summarized_embeddings(
                sentences, ratio, algorithm, use_first, num_sentences)

            if aggregate is not None:
                assert aggregate in [
                    'mean', 'median', 'max', 'min'], "aggregate must be mean, min, max, or median"
                embeddings = self.aggregate_map[aggregate](embeddings, axis=0)

            return embeddings

        return None
    
    def run(self,
            body: str,
            ratio: float = 0.2,
            min_length: int = 40,
            max_length: int = 600,
            use_first: bool = True,
            algorithm: str = 'kmeans',
            num_sentences: int = None,
            return_as_list: bool = False) -> Union[List, str]:
        """
        Preprocesses the sentences, runs the clusters to find the centroids, then combines the sentences.
        :param body: The raw string body to process
        :param ratio: Ratio of sentences to use
        :param min_length: Minimum length of sentence candidates to utilize for the summary.
        :param max_length: Maximum length of sentence candidates to utilize for the summary
        :param use_first: Whether or not to use the first sentence
        :param algorithm: Which clustering algorithm to use. (kmeans, gmm)
        :param num_sentences: Number of sentences to use (overrides ratio).
        :param return_as_list: Whether or not to return sentences as list.
        :return: A summary sentence
        """
        sentences = self.sentence_handler(body, min_length, max_length)

        if sentences:
            sentences = self.__run_clusters(sentences, ratio, algorithm, use_first, num_sentences)
            
        #Vietnamese spaCy language package also have _ characters between compound words, so we have to replace it
        if return_as_list:
            return sentences.replace('_', ' ')
        else:
            return ' '.join(sentences).replace('_',' ')

    def __call__(self,
                body: str,
                ratio: float = 0.2,
                min_length: int = 40,
                max_length: int = 600,
                use_first: bool = True,
                algorithm: str = 'gmm', #kmeans
                num_sentences: int = None,
                return_as_list: bool = False) -> str:
        """
        (utility that wraps around the run function)
        Preprocesses the sentences, runs the clusters to find the centroids, then combines the sentences.
        :param body: The raw string body to process.
        :param ratio: Ratio of sentences to use.
        :param min_length: Minimum length of sentence candidates to utilize for the summary.
        :param max_length: Maximum length of sentence candidates to utilize for the summary.
        :param use_first: Whether or not to use the first sentence.
        :param algorithm: Which clustering algorithm to use. (kmeans, gmm)
        :param Number of sentences to use (overrides ratio).
        :param return_as_list: Whether or not to return sentences as list.
        :return: A summary sentence.
        """
        return self.run(
            body, ratio, min_length, max_length, algorithm=algorithm, use_first=use_first, num_sentences=num_sentences,
            return_as_list=return_as_list)

class Summarizer(ModelProcessor):
    def __init__(self,
        model: str = 'bert-base-multilingual-uncased',
        custom_model: PreTrainedModel = None,
        custom_tokenizer: PreTrainedTokenizer = None,
        hidden: Union[List[int], int] = -2,
        reduce_option: str = 'mean',
        sentence_handler: SentenceHandler = SentenceHandler(),
        random_state: int = 12345,
        hidden_concat: bool = False):
        """
        This is the main Bert Summarizer class.
        :param model: This parameter is associated with the inherit string parameters from the transformers library.
        :param custom_model: If you have a pre-trained model, you can add the model class here.
        :param custom_tokenizer: If you have a custom tokenizer, you can add the tokenizer here.
        :param hidden: This signifies which layer of the BERT model you would like to use as embeddings.
        :param reduce_option: Given the output of the bert model, this param determines how you want to reduce results.
        :param random_state: The random state to reproduce summarizations.
        :param hidden_concat: Whether or not to concat multiple hidden layers.
        """
        super(Summarizer, self).__init__(
            model, custom_model, custom_tokenizer, hidden, reduce_option, sentence_handler, random_state, hidden_concat
        )

class TransformerSummarizer(ModelProcessor):
    """
    Newer style that has keywords for models and tokenizers, but allows the user to change the type.
    """
    MODEL_DICT = {
                    'Bert': (BertModel, BertTokenizer),
                    'GPT2': (GPT2Model, GPT2Tokenizer),
                    'OpenAIGPT': (OpenAIGPTModel, OpenAIGPTTokenizer),
                    'CTRL': (CTRLModel, CTRLTokenizer),
                    'TransfoXL': (TransfoXLModel, TransfoXLTokenizer),
                    'XLNet': (XLNetModel, XLNetTokenizer),
                    'XLM': (XLMModel, XLMTokenizer),
                    'DistilBert': (DistilBertModel, DistilBertTokenizer),
                    'ALBERT': (AlbertModel, AlbertTokenizer),
                    'phoBERT': (AutoModel, AutoTokenizer),
    }
    def __init__(self,
                transformer_type: str = 'Bert',
                transformer_model_key: str = 'bert-base-multilingual-uncased',
                transformer_tokenizer_key: str = None,
                hidden: Union[List[int], int] = -2,
                reduce_option: str = 'mean',
                sentence_handler: SentenceHandler = SentenceHandler(),
                random_state: int = 12345,
                hidden_concat: bool = False):
        """
        :param transformer_type: The Transformer type, such as Bert, GPT2, DistilBert, etc.
        :param transformer_model_key: The transformer model key. This is the directory for the model.
        :param transformer_tokenizer_key: The transformer tokenizer key. This is the tokenizer directory.
        :param hidden: The hidden output layers to use for the summarization.
        :param reduce_option: The reduce option, such as mean, max, min, median, etc.
        :param sentence_handler: The sentence handler class to process the raw text.
        :param random_state: The random state to use.
        :param hidden_concat: Deprecated hidden concat option.
        """
        try:
            self.MODEL_DICT['Roberta'] = (RobertaModel, RobertaTokenizer)
            self.MODEL_DICT['Albert'] = (AlbertModel, AlbertTokenizer)
            self.MODEL_DICT['Camembert'] = (CamembertModel, CamembertTokenizer)
            self.MODEL_DICT['Bart'] = (BartModel, BartTokenizer)
            self.MODEL_DICT['Longformer'] = (LongformerModel, LongformerTokenizer)
        except Exception:
            pass  # older transformer version

        model_clz, tokenizer_clz = self.MODEL_DICT[transformer_type]
        model = model_clz.from_pretrained(
            transformer_model_key, output_hidden_states=True)

        tokenizer = tokenizer_clz.from_pretrained(
            transformer_tokenizer_key if transformer_tokenizer_key is not None else transformer_model_key
        )

        super().__init__(
            None, model, tokenizer, hidden, reduce_option, sentence_handler, random_state, hidden_concat
        )

In [8]:
def rouge_dist(hypotheses, references):
    rouge = PyRouge(rouge_n=(1, 2), rouge_l=True, rouge_su=True, skip_gap=4)
    scores = rouge.evaluate(hypotheses, references)
    return scores

In [9]:
import os

def loadtxt(path, ref=False):
    bodyinput=[]
    # Change the directory
    os.chdir(path)
    # Read text File  
    def read_text_file(file_path):
        with open(file_path, 'r', encoding = 'utf-8') as f:
            ftext = f.read()
            ftext = ftext.replace('\ufeff ','').replace('\ufeff','').replace('\n','')
            return ftext
        
    # iterate through all file
    for file in os.listdir():
        if file.endswith(".txt"):
            file_path = f"{path}\{file}"
            if ref:
                bodyinput.append([read_text_file(file_path)])
            else: 
                bodyinput.append(read_text_file(file_path))
    return bodyinput

# Analyzing

In [10]:
model_dict = {
                'bert-base-multilingual-uncased': (BertModel, BertTokenizer),
                'bert-large-uncased': (BertModel, BertTokenizer),
                'gpt2': (GPT2Model, GPT2Tokenizer),
                'facebook/bart-large': (BartModel, BartTokenizer),
                'openai-gpt': (OpenAIGPTModel, OpenAIGPTTokenizer),
                'ctrl': (CTRLModel, CTRLTokenizer),
                'transfo-xl-wt103': (TransfoXLModel, TransfoXLTokenizer), 
                'xlnet-large-cased': (XLNetModel, XLNetTokenizer),
                'xlm-mlm-enfr-1024': (XLMModel, XLMTokenizer),
                'distilbert-base-uncased': (DistilBertModel, DistilBertTokenizer),
                'albert-large-v2': (AlbertModel, AlbertTokenizer),
                'allenai/scibert_scivocab_uncased': (AutoModel, AutoTokenizer),
                'vinai/phobert-large': (AutoModel, AutoTokenizer)
}

In [11]:
category = ['boKHCN', 'Chinh Tri', 'khoahoc_giaoduc', 'kinhte', 'Van Hoa', 'Xa Hoi']

In [12]:
idx = category[0]
pathbody = 'E:/TextSummarization/donvanban/Plaintext/' + idx 
pathref = 'E:/TextSummarization/donvanban/Summary_manual/' + idx 
rootbody = loadtxt(pathbody)
refbody = loadtxt(pathref, ref=True)

In [14]:
res = {'Rouge1_p':[], 'Rouge2_p':[], 'RougeL_p':[], 'RougeSU4_p': [],
        'Rouge1_r':[], 'Rouge2_r':[], 'RougeL_r':[], 'RougeSU4_r': [],
        'Rouge1_f':[], 'Rouge2_f':[], 'RougeL_f':[], 'RougeSU4_f': [],
      }

for idx in model_dict.keys():    
    model = Summarizer(model = idx, sentence_handler=CoreferenceHandler())
    all_result = []
    for jdx in range(len(rootbody)):
        result = model(rootbody[jdx], ratio=2*len(refbody[jdx])/len(rootbody[jdx]))
        all_result.append(result)
    r_rouge = rouge_dist(all_result, refbody)
    for ind in ['p', 'r', 'f']:
        res['Rouge1_'+ind].append(r_rouge['rouge-1'][ind])
        res['Rouge2_'+ind].append(r_rouge['rouge-2'][ind])
        res['RougeL_'+ind].append(r_rouge['rouge-l'][ind])
        res['RougeSU4_'+ind].append(r_rouge['rouge-su4'][ind])

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor di

In [15]:
#score f, precision p, recall r
df = pd.DataFrame(res, index = model_dict.keys())
df

,Rouge1_p,Rouge2_p,RougeL_p,RougeSU4_p,Rouge1_r,Rouge2_r,RougeL_r,RougeSU4_r,Rouge1_f,Rouge2_f,RougeL_f,RougeSU4_f
bert-base-multilingual-uncased,0.589949,0.468815,0.471425,0.440070,0.531650,0.419202,0.422021,0.392686,0.559284,0.442622,0.445357,0.415030
bert-large-uncased,0.577423,0.442054,0.440485,0.413317,0.443022,0.337761,0.336407,0.315294,0.501372,0.382934,0.381475,0.357712
gpt2,0.592837,0.456343,0.459239,0.423710,0.477464,0.366082,0.367362,0.339460,0.528932,0.406259,0.408194,0.376934
facebook/bart-large,0.559149,0.418020,0.430839,0.388407,0.472508,0.358028,0.366000,0.333033,0.512190,0.385705,0.395781,0.358595
openai-gpt,0.572291,0.433563,0.442683,0.404568,0.466779,0.355695,0.361708,0.331607,0.514178,0.390787,0.398120,0.364472
ctrl,0.572195,0.439382,0.430547,0.413148,0.516807,0.388970,0.382233,0.363626,0.543093,0.412642,0.404954,0.386809
transfo-xl-wt103,0.599472,0.454298,0.446087,0.423647,0.411946,0.304241,0.302832,0.282794,0.488324,0.364427,0.360758,0.339179
xlnet-large-cased,0.585887,0.463378,0.466526,0.435221,0.532594,0.418054,0.419548,0.392126,0.557971,0.439551,0.441791,0.412551
xlm-mlm-enfr-1024,0.573055,0.437195,0.430924,0.407219,0.425831,0.316865,0.318507,0.294447,0.488594,0.367429,0.366284,0.341771
distilbert-base-uncased,0.562064,0.423952,0.431258,0.395780,0.495183,0.379124,0.383464,0.354335,0.526508,0.400287,0.405959,0.373912


# Analyzing others

In [22]:
#check utf-8 encode errors on datasets
#plain vh020304, summary ct17, vh11
'''
os.chdir('E:/TextSummarization/donvanban/Summary_manual/Xa Hoi')
for file in os.listdir():
        print(file)
        if file.endswith(".txt"):
            file_path = f"{'E:/TextSummarization/donvanban/Summary_manual/Xa Hoi'}\{file}"
            with open(file_path, 'r', encoding = 'utf-8') as f:
                ftext = f.read()
                ftext = ftext.replace('\ufeff ','').replace('\ufeff','').replace('\n','')
'''

XH01.txt
XH02.txt
XH03.txt
XH04.txt
XH05.txt
XH06.txt
XH07.txt
XH08.txt
XH09.txt
XH10.txt
XH11.txt
XH12.txt
XH13.txt
XH14.txt
XH15.txt
XH16.txt
XH17.txt
XH18.txt
XH19.txt
XH20.txt
XH21.txt
XH22.txt
XH23.txt
XH24.txt
XH25.txt
XH26.txt
XH27.txt
XH28.txt
XH29.txt
XH30.txt
XH31.txt
XH32.txt
XH33.txt
XH34.txt
XH35.txt


In [21]:
#chinh tri
idx = category[1]

pathbody = 'E:/TextSummarization/donvanban/Plaintext/' + idx 
pathref = 'E:/TextSummarization/donvanban/Summary_manual/' + idx 
rootbody = loadtxt(pathbody)
refbody = loadtxt(pathref, ref=True)

res = {'Rouge1_p':[], 'Rouge2_p':[], 'RougeL_p':[], 'RougeSU4_p': [],
        'Rouge1_r':[], 'Rouge2_r':[], 'RougeL_r':[], 'RougeSU4_r': [],
        'Rouge1_f':[], 'Rouge2_f':[], 'RougeL_f':[], 'RougeSU4_f': [],
      }

for idx in model_dict.keys():    
    model = Summarizer(model = idx, sentence_handler=CoreferenceHandler())
    all_result = []
    for jdx in range(len(rootbody)):
        result = model(rootbody[jdx], ratio=2*len(refbody[jdx])/len(rootbody[jdx]))
        all_result.append(result)
    r_rouge = rouge_dist(all_result, refbody)
    for ind in ['p', 'r', 'f']:
        res['Rouge1_'+ind].append(r_rouge['rouge-1'][ind])
        res['Rouge2_'+ind].append(r_rouge['rouge-2'][ind])
        res['RougeL_'+ind].append(r_rouge['rouge-l'][ind])
        res['RougeSU4_'+ind].append(r_rouge['rouge-su4'][ind])
df = pd.DataFrame(res, index = model_dict.keys())
df

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor di

,Rouge1_p,Rouge2_p,RougeL_p,RougeSU4_p,Rouge1_r,Rouge2_r,RougeL_r,RougeSU4_r,Rouge1_f,Rouge2_f,RougeL_f,RougeSU4_f
bert-base-multilingual-uncased,0.606898,0.488426,0.511353,0.464321,0.460991,0.372269,0.387233,0.352674,0.523977,0.422509,0.440721,0.400868
bert-large-uncased,0.584729,0.466408,0.493596,0.447245,0.423522,0.342373,0.357176,0.327926,0.491238,0.394879,0.414449,0.378403
gpt2,0.591884,0.470366,0.495039,0.446783,0.428530,0.347310,0.361333,0.329626,0.497131,0.399578,0.417748,0.379365
facebook/bart-large,0.571642,0.449727,0.474903,0.430690,0.441181,0.354861,0.371311,0.339761,0.498009,0.396701,0.416766,0.379860
openai-gpt,0.603978,0.479673,0.500607,0.456440,0.413490,0.329500,0.342801,0.312312,0.490903,0.390652,0.406940,0.370865
ctrl,0.570475,0.447075,0.464067,0.426458,0.449808,0.346983,0.360848,0.329545,0.503006,0.390720,0.406000,0.371790
transfo-xl-wt103,0.637110,0.514270,0.536534,0.490145,0.418280,0.342160,0.355364,0.324859,0.505008,0.410921,0.427549,0.390742
xlnet-large-cased,0.581520,0.467159,0.483462,0.448211,0.457777,0.366858,0.380432,0.350807,0.512282,0.410977,0.425803,0.393572
xlm-mlm-enfr-1024,0.581030,0.458219,0.480850,0.434891,0.437265,0.350573,0.363914,0.333355,0.498999,0.397232,0.414288,0.377413
distilbert-base-uncased,0.613016,0.491844,0.519021,0.472632,0.437599,0.359126,0.378010,0.345194,0.510664,0.415135,0.437432,0.398984


In [18]:
#khoahockythuat
idx = category[2]

pathbody = 'E:/TextSummarization/donvanban/Plaintext/' + idx 
pathref = 'E:/TextSummarization/donvanban/Summary_manual/' + idx 
rootbody = loadtxt(pathbody)
refbody = loadtxt(pathref, ref=True)

res = {'Rouge1_p':[], 'Rouge2_p':[], 'RougeL_p':[], 'RougeSU4_p': [],
        'Rouge1_r':[], 'Rouge2_r':[], 'RougeL_r':[], 'RougeSU4_r': [],
        'Rouge1_f':[], 'Rouge2_f':[], 'RougeL_f':[], 'RougeSU4_f': [],
      }

for idx in model_dict.keys():    
    model = Summarizer(model = idx, sentence_handler=CoreferenceHandler())
    all_result = []
    for jdx in range(len(rootbody)):
        result = model(rootbody[jdx], ratio=2*len(refbody[jdx])/len(rootbody[jdx]))
        all_result.append(result)
    r_rouge = rouge_dist(all_result, refbody)
    for ind in ['p', 'r', 'f']:
        res['Rouge1_'+ind].append(r_rouge['rouge-1'][ind])
        res['Rouge2_'+ind].append(r_rouge['rouge-2'][ind])
        res['RougeL_'+ind].append(r_rouge['rouge-l'][ind])
        res['RougeSU4_'+ind].append(r_rouge['rouge-su4'][ind])
df = pd.DataFrame(res, index = model_dict.keys())
df

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
c:\users\technical\appdata\local\programs\python\python37\lib\site-packages\transformers\configuration_transfo_xl.py:146: FutureWarning: The config parameter `tie_weight` is deprecated. Please use `tie_word_embeddings` instead.
  FutureWarning,
c:\users\technical\appdata\local\programs\python\python37\lib\site-packages\transformers\configuration_xlnet.py:212: FutureWarning: This config doesn't use attenti

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


,Rouge1_p,Rouge2_p,RougeL_p,RougeSU4_p,Rouge1_r,Rouge2_r,RougeL_r,RougeSU4_r,Rouge1_f,Rouge2_f,RougeL_f,RougeSU4_f
bert-base-multilingual-uncased,0.536254,0.368117,0.411873,0.349936,0.357484,0.245265,0.273270,0.230621,0.428990,0.294388,0.328552,0.278018
bert-large-uncased,0.543157,0.354487,0.399662,0.332533,0.313515,0.207422,0.228703,0.193525,0.397557,0.261709,0.290926,0.244663
gpt2,0.584382,0.411494,0.454490,0.389178,0.305186,0.210687,0.232031,0.195871,0.400970,0.278685,0.307218,0.260589
facebook/bart-large,0.564344,0.378987,0.405870,0.356827,0.310637,0.207347,0.221083,0.192267,0.400708,0.268044,0.286245,0.249888
openai-gpt,0.570364,0.392951,0.418475,0.372536,0.318832,0.216094,0.232056,0.202027,0.409022,0.278845,0.298555,0.261981
ctrl,0.486931,0.319998,0.358052,0.305114,0.367720,0.250466,0.272123,0.236195,0.419012,0.280995,0.309229,0.266267
transfo-xl-wt103,0.583542,0.406037,0.456251,0.379149,0.262237,0.180220,0.199831,0.164662,0.361859,0.249638,0.277932,0.229607
xlnet-large-cased,0.525853,0.352307,0.391537,0.331034,0.352453,0.244075,0.266600,0.227216,0.422036,0.288370,0.317210,0.269471
xlm-mlm-enfr-1024,0.565920,0.388310,0.432547,0.367681,0.283499,0.191423,0.211416,0.177515,0.377759,0.256433,0.284015,0.239433
distilbert-base-uncased,0.539658,0.354672,0.397923,0.333094,0.315460,0.208399,0.230405,0.192865,0.398169,0.262536,0.291833,0.244286


In [12]:
idx = category[3]

pathbody = 'E:/TextSummarization/donvanban/Plaintext/' + idx 
pathref = 'E:/TextSummarization/donvanban/Summary_manual/' + idx 
rootbody = loadtxt(pathbody)
refbody = loadtxt(pathref, ref=True)

res = {'Rouge1_p':[], 'Rouge2_p':[], 'RougeL_p':[], 'RougeSU4_p': [],
        'Rouge1_r':[], 'Rouge2_r':[], 'RougeL_r':[], 'RougeSU4_r': [],
        'Rouge1_f':[], 'Rouge2_f':[], 'RougeL_f':[], 'RougeSU4_f': [],
      }

for idx in model_dict.keys():    
    model = Summarizer(model = idx, sentence_handler=CoreferenceHandler())
    all_result = []
    for jdx in range(len(rootbody)):
        result = model(rootbody[jdx], ratio=2*len(refbody[jdx])/len(rootbody[jdx]))
        all_result.append(result)
    r_rouge = rouge_dist(all_result, refbody)
    for ind in ['p', 'r', 'f']:
        res['Rouge1_'+ind].append(r_rouge['rouge-1'][ind])
        res['Rouge2_'+ind].append(r_rouge['rouge-2'][ind])
        res['RougeL_'+ind].append(r_rouge['rouge-l'][ind])
        res['RougeSU4_'+ind].append(r_rouge['rouge-su4'][ind])
df = pd.DataFrame(res, index = model_dict.keys())
df

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
c:\users\technical\appdata\local\programs\python\python37\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual flo

,Rouge1_p,Rouge2_p,RougeL_p,RougeSU4_p,Rouge1_r,Rouge2_r,RougeL_r,RougeSU4_r,Rouge1_f,Rouge2_f,RougeL_f,RougeSU4_f
bert-base-multilingual-uncased,0.530388,0.356513,0.405619,0.333932,0.299726,0.200465,0.226252,0.185960,0.383010,0.256629,0.290477,0.238888
bert-large-uncased,0.492336,0.335471,0.383629,0.317570,0.276358,0.186792,0.212648,0.174713,0.354006,0.239968,0.273624,0.225414
gpt2,0.569050,0.409568,0.450531,0.388871,0.280465,0.198937,0.219082,0.185618,0.375741,0.267798,0.294806,0.251289
facebook/bart-large,0.518434,0.340215,0.400308,0.324933,0.286019,0.190926,0.217445,0.179552,0.368653,0.244590,0.281812,0.231295
openai-gpt,0.570235,0.406665,0.456003,0.385390,0.300957,0.214085,0.238028,0.198954,0.393980,0.280502,0.312786,0.262431
ctrl,0.560080,0.408440,0.443820,0.393660,0.302701,0.212283,0.233710,0.201295,0.393001,0.279368,0.306186,0.266379
transfo-xl-wt103,0.529068,0.351818,0.417442,0.338171,0.252087,0.173870,0.200028,0.164086,0.341472,0.232726,0.270459,0.220959
xlnet-large-cased,0.557770,0.394704,0.439757,0.380134,0.324983,0.228656,0.253074,0.217262,0.410683,0.289564,0.321264,0.276495
xlm-mlm-enfr-1024,0.533898,0.359918,0.417652,0.339807,0.272771,0.183817,0.211893,0.171892,0.361069,0.243350,0.281148,0.228299
distilbert-base-uncased,0.553853,0.388896,0.441077,0.372155,0.304812,0.213395,0.241322,0.201049,0.393217,0.275576,0.311963,0.261064


In [12]:
idx = category[4]

pathbody = 'E:/TextSummarization/donvanban/Plaintext/' + idx 
pathref = 'E:/TextSummarization/donvanban/Summary_manual/' + idx 
rootbody = loadtxt(pathbody)
refbody = loadtxt(pathref, ref=True)

res = {'Rouge1_p':[], 'Rouge2_p':[], 'RougeL_p':[], 'RougeSU4_p': [],
        'Rouge1_r':[], 'Rouge2_r':[], 'RougeL_r':[], 'RougeSU4_r': [],
        'Rouge1_f':[], 'Rouge2_f':[], 'RougeL_f':[], 'RougeSU4_f': [],
      }

for idx in model_dict.keys():    
    model = Summarizer(model = idx, sentence_handler=CoreferenceHandler())
    all_result = []
    for jdx in range(len(rootbody)):
        result = model(rootbody[jdx], ratio=2*len(refbody[jdx])/len(rootbody[jdx]))
        all_result.append(result)
    r_rouge = rouge_dist(all_result, refbody)
    for ind in ['p', 'r', 'f']:
        res['Rouge1_'+ind].append(r_rouge['rouge-1'][ind])
        res['Rouge2_'+ind].append(r_rouge['rouge-2'][ind])
        res['RougeL_'+ind].append(r_rouge['rouge-l'][ind])
        res['RougeSU4_'+ind].append(r_rouge['rouge-su4'][ind])
df = pd.DataFrame(res, index = model_dict.keys())
df

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor di

,Rouge1_p,Rouge2_p,RougeL_p,RougeSU4_p,Rouge1_r,Rouge2_r,RougeL_r,RougeSU4_r,Rouge1_f,Rouge2_f,RougeL_f,RougeSU4_f
bert-base-multilingual-uncased,0.419959,0.260637,0.318426,0.246730,0.251632,0.160160,0.189027,0.151359,0.314701,0.198403,0.237228,0.187620
bert-large-uncased,0.408568,0.254353,0.307670,0.246895,0.208488,0.126279,0.152615,0.120526,0.276090,0.168769,0.204026,0.161979
gpt2,0.433021,0.277941,0.332966,0.265527,0.215373,0.139819,0.165171,0.132079,0.287667,0.186047,0.220808,0.176408
facebook/bart-large,0.421589,0.270862,0.328721,0.260702,0.225046,0.150777,0.175961,0.143450,0.293448,0.193719,0.229222,0.185067
openai-gpt,0.443905,0.292327,0.340738,0.282514,0.237590,0.161473,0.181809,0.153630,0.309518,0.208034,0.237105,0.199029
ctrl,0.407358,0.253398,0.304184,0.241371,0.240232,0.144653,0.171588,0.136343,0.302229,0.184171,0.219409,0.174255
transfo-xl-wt103,0.438544,0.266815,0.335239,0.255635,0.218167,0.138108,0.165696,0.131283,0.291379,0.182007,0.221776,0.173476
xlnet-large-cased,0.381087,0.219418,0.273056,0.210942,0.245915,0.141625,0.172453,0.135256,0.298931,0.172141,0.211396,0.164826
xlm-mlm-enfr-1024,0.410395,0.244495,0.307248,0.234934,0.220099,0.130600,0.161009,0.124493,0.286530,0.170255,0.211293,0.162746
distilbert-base-uncased,0.421393,0.253721,0.314517,0.246651,0.229169,0.145959,0.174464,0.139197,0.296883,0.185313,0.224433,0.177962


In [13]:
idx = category[5]

pathbody = 'E:/TextSummarization/donvanban/Plaintext/' + idx 
pathref = 'E:/TextSummarization/donvanban/Summary_manual/' + idx 
rootbody = loadtxt(pathbody)
refbody = loadtxt(pathref, ref=True)

res = {'Rouge1_p':[], 'Rouge2_p':[], 'RougeL_p':[], 'RougeSU4_p': [],
        'Rouge1_r':[], 'Rouge2_r':[], 'RougeL_r':[], 'RougeSU4_r': [],
        'Rouge1_f':[], 'Rouge2_f':[], 'RougeL_f':[], 'RougeSU4_f': [],
      }

for idx in model_dict.keys():    
    model = Summarizer(model = idx, sentence_handler=CoreferenceHandler())
    all_result = []
    for jdx in range(len(rootbody)):
        result = model(rootbody[jdx], ratio=2*len(refbody[jdx])/len(rootbody[jdx]))
        all_result.append(result)
    r_rouge = rouge_dist(all_result, refbody)
    for ind in ['p', 'r', 'f']:
        res['Rouge1_'+ind].append(r_rouge['rouge-1'][ind])
        res['Rouge2_'+ind].append(r_rouge['rouge-2'][ind])
        res['RougeL_'+ind].append(r_rouge['rouge-l'][ind])
        res['RougeSU4_'+ind].append(r_rouge['rouge-su4'][ind])
df = pd.DataFrame(res, index = model_dict.keys())
df

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
c:\users\technical\appdata\local\programs\python\python37\lib\site-packages\transformers\configuration_transfo_xl.py:146: FutureWarning: The config parameter `tie_weight` is deprecated. Please use `tie_word_embeddings` instead.
  FutureWarning,
c:\users\technical\appdata\local\programs\python\python37\lib\site-packages\transformers\configuration_xlnet.py:212: FutureWarning: This config doesn't use attenti

,Rouge1_p,Rouge2_p,RougeL_p,RougeSU4_p,Rouge1_r,Rouge2_r,RougeL_r,RougeSU4_r,Rouge1_f,Rouge2_f,RougeL_f,RougeSU4_f
bert-base-multilingual-uncased,0.500717,0.335891,0.377074,0.319064,0.304166,0.211028,0.231594,0.198776,0.378443,0.259207,0.286948,0.244950
bert-large-uncased,0.503477,0.344400,0.380821,0.328923,0.258243,0.176876,0.195496,0.165626,0.341384,0.233719,0.258361,0.220315
gpt2,0.540692,0.380299,0.423641,0.365884,0.267464,0.189232,0.209020,0.178608,0.357890,0.252715,0.279927,0.240039
facebook/bart-large,0.531781,0.364206,0.416920,0.350735,0.294335,0.207849,0.230653,0.197494,0.378934,0.264659,0.296998,0.252698
openai-gpt,0.487840,0.317110,0.370046,0.306068,0.267715,0.175316,0.201075,0.165604,0.345712,0.225798,0.260565,0.214921
ctrl,0.507409,0.356920,0.376180,0.341621,0.305062,0.211551,0.223330,0.199203,0.381038,0.265649,0.280269,0.251660
transfo-xl-wt103,0.525901,0.352383,0.398021,0.334715,0.245627,0.168283,0.186899,0.156607,0.334856,0.227786,0.254359,0.213378
xlnet-large-cased,0.519272,0.374568,0.402207,0.353530,0.294641,0.215823,0.229381,0.200726,0.375959,0.273854,0.292148,0.256065
xlm-mlm-enfr-1024,0.524133,0.371204,0.409127,0.353783,0.306791,0.223627,0.241264,0.209859,0.387038,0.279109,0.303533,0.263446
distilbert-base-uncased,0.504549,0.329222,0.376833,0.316757,0.288143,0.195924,0.215930,0.185842,0.366806,0.245656,0.274543,0.234249


# Sample

In [ ]:
'''
Summarizer(
    model: This gets used by the hugging face bert library to load the model, you can supply a custom trained model here
    custom_model: If you have a pre-trained model, you can add the model class here.
    custom_tokenizer:  If you have a custom tokenizer, you can add the tokenizer here.
    hidden: Needs to be negative, but allows you to pick which layer you want the embeddings to come from.
    reduce_option: It can be 'mean', 'median', or 'max'. This reduces the embedding layer for pooling.
    sentence_handler: The handler to process sentences. If want to use coreference, instantiate and pass CoreferenceHandler instance)
model(body: str # The string body that you want to summarize
    ratio: float # The ratio of sentences that you want for the final summary
    min_length: int # Parameter to specify to remove sentences that are less than 40 characters
    max_length: int # Parameter to specify to remove sentences greater than the max length,
    num_sentences: Number of sentences to use. Overrides ratio if supplied)
'''

In [104]:
body = '''
Shiba Inu (柴犬 (sài khuyển)) là loại chó nhỏ nhất trong sáu giống chó nguyên thủy và riêng biệt đến từ Nhật Bản. Chúng là một giống chó nhỏ, nhanh nhẹn và thích hợp với địa hình miền núi, Shiba Inu ban đầu được nuôi để săn bắt. Nó gần giống nhưng nhỏ hơn so với giống Akita Inu. Đây là một trong số ít giống chó cổ xưa vẫn còn tồn tại cho đến ngày nay.
Shiba là một trong sáu giống chó điển hình của Nhật Bản, cũng như Hokkaido, Kishu, Shikoku, Kai và Akita. Trong những giống chó này, Shiba là nhỏ nhất.
Inu hoặc ken (犬 - Hán Việt: khuyển) trong tiếng Nhật có nghĩa là con chó, nhưng nguồn gốc của từ "Shiba" vẫn chưa rõ. Từ Shiba (柴 - Hán Việt: sài) có nghĩa là cây bụi trong tiếng Nhật, đề cập đến một loại cây hoặc cây bụi có lá chuyển sang màu đỏ vào mùa thu. Điều này khiến cho một số người tin rằng Shiba được đặt tên như thế là vì loài chó này được sử dụng để săn mồi trong các bụi cây, hoặc có thể là do màu sắc phổ biến nhất của Shiba Inu là màu đỏ tương tự như của các cây bụi. Tuy nhiên, trong một phương ngữ Nagano cổ, từ Shiba cũng có ý nghĩa là nhỏ, do đó cái tên có thể nói đến tầm vóc nhỏ bé của con chó. Do đó, Shiba Inu đôi khi được dịch là "Little Brushwood Dog", tức "Chó bụi nhỏ".
Khung hình của Shiba nhỏ gọn với cơ bắp phát triển tốt. Con đực có chiều cao từ 35 đến 43 cm (14 đến 17 in). Đối với con cái là 33 đến 41 cm (13 đến 16 in). Trọng lượng trung bình ở kích thước tương thích là khoảng 10 kg (22 lb) đối với con đực và 8 kg (18 lb) đối với con cái. Xương vừa phải.
Lớp lông: Có hai lớp lông với lớp ngoài cứng và thẳng cùng một lớp trong mềm mại và dày. Lông mao ngắn và thậm chí trên mặt, tai và chân giống cáo. Lông bảo vệ xù ra khỏi cơ thể chiều dài khoảng 4 đến 5 cm (1 1⁄2 đến 2 in) ở vai. Lông đuôi hơi dài và xù ra. Shiba có thể có màu đỏ, đen và nâu, hoặc màu vừng (màu đỏ với những sợi ngã sang đen), với một lớp lông lót màu kem, màu da bò, hoặc màu xám. Nó cũng có thể có màu trắng (kem), mặc dù màu này được xem là một"lỗi nghiêm trọng"bởi Hiệp hội chó giống Mỹ và không bao giờ được nuôi trong các chương trình. Ngược lại, một lớp lông màu trắng (kem) là hoàn toàn chấp nhận được theo tiêu chuẩn giống chó Anh.
Urajiro (màu kem trắng) có ở các bộ phận sau trên tất cả các vùng lông: ở hai bên mõm, trên má, bên trong tai, trên hàm dưới và ở chỗ cổ họng, bên trong chân, trên bụng, xung quanh các lỗ thông hơi và phía vùng bụng của đuôi. Màu đỏ: thương ở trên cổ họng, chóp ngực và ngực. Đen và màu vừng: thường là một dấu tam giác trên cả hai bên của chóp ngực.
Shiba có xu hướng thể hiện tính tự lập và đôi khi còn hung hăng. Shiba Inu tốt nhất nên được nuôi trong một gia đình mà không có những con chó nhỏ khác hay trẻ em, nhưng huấn luyện vâng lời vẫn có thể được và xã hội sớm có thể làm cho tất cả trở nên ngoan ngoãn. Giống chó cũng tương tác khá tốt với mèo.
Một tinh thần mạnh dạn, một bản chất tốt đẹp và sự thẳng thắn không lẫn lộn mang lại phẩm giá và vẻ đẹp tự nhiên. Shiba có tính chất tự lập và có thể dè dặt đối với người lạ nhưng lại trung thành và tình cảm với những người có được sự tôn trọng của nó. Nó có thể hung dữ với những con chó khác.
Shiba là một giống chó tương đối khó tính và cảm thấy rất cần thiết khi giữ chính nó thật sạch. Nó thường liếm bàn chân giống như mèo, thường di chuyển theo cách riêng của mình để giữ bộ lông sạch sẽ, nhưng lại cực kỳ thích bơi lội và chơi đùa trong các vũng nước. Vì bản chất khó tính và đầy kiêu hãnh vốn có, Shiba con rất dễ dạy dỗ và trong nhiều trường hợp sẽ tự dạy dỗ chính mình. Chỉ cần chủ đơn giản là đặt chúng ra ngoài sau giờ ăn và ngủ thì có thể nói là đã đủ để dạy Shiba phương pháp thích hợp để đi vệ sinh.
Một đặc điểm giúp phân biệt giống chó này là "Shiba scream". Khi đủ kích động hay không vui, nó sẽ phát ra một tiếng thét lớn và cao. Điều này có thể xảy ra khi nó cố gắng để xử lý con chó theo một cách mà nó cho là không thể chấp nhận được. Các động vật khác cũng có thể phát ra âm thanh tương tự như trong những lúc vui, chẳng hạn như sự trở lại của chủ nhân sau khi vắng mặt lâu ngày hay sự xuất hiện của một người khách yêu thích,...
Thí nghiệm phân tích DNA gần đây đã khẳng định rằng loài chó mõm nhọn châu Á này là một trong những giống chó lâu đời nhất, đã sống từ thế kỷ thứ 3 trước Công nguyên.
Ban đầu, Shiba Inu được nuôi để săn và bắt các con vật nhỏ, chẳng hạn như các loài chim và thỏ. Dù đã có nhiều nỗ lực để bảo tồn giống, Shiba gần bị tuyệt chủng trong Chiến tranh thế giới thứ hai do tình trạng thiếu thực phẩm cộng thêm dịch bệnh chó sau chiến tranh. Tất cả những con chó sau này được tạo ra chỉ từ ba dòng máu còn sống sót. Những dòng máu đó là Shinshu Shiba từ Nagano, Mino Shiba từ Gifu, và San'in Shiba từ Tottori và Shimane. Shinshu Shiba sở hữu một lớp lông tơ rắn, với một lớp lông dày bảo vệ, nhỏ và có màu đỏ. Mino Shiba thường có đôi tai dày, nhọn và sở hữu một cái đuôi hình lưỡi liềm, chứ không phải là đuôi cuộn tròn thường được tìm thấy trên Shiba hiện nay. San'in Shiba thì lớn hơn so với hầu hết các giống Shiba hiện nay, và thường có màu đen, không có dấu sẫm và trắng thường được tìm thấy trên Shiba đen - sẫm hiện nay. Khi nghiên cứu về chó Nhật được chính thức hóa trong đầu và giữa thế kỷ 20, ba chủng này đã được kết hợp thành một giống tổng thể, Shiba Inu. Các tiêu chuẩn giống Nhật Bản đầu tiên cho Shiba, tiêu chuẩn Nippo, được xuất bản vào năm 1934. Vào tháng 12 năm 1936, các Shiba Inu được công nhận là Di tích tự nhiên của Nhật Bản thông qua Đạo luật văn hóa, phần lớn là do những nỗ lực của Nippo (Nihon Ken Hozonkai) - Hiệp hội Bảo tồn Chó Nhật Bản.
Năm 1954, một gia đình phục vụ vũ trang mang con Shiba Inu đầu tiên đến Hoa Kỳ. Vào năm 1979, lứa đầu tiên được ghi nhận sinh ra tại Hoa Kỳ. Shiba đã được công nhận bởi Hiệp hội chó giống Mỹ vào năm 1992 và được bổ sung vào nhóm AKC (nhóm phi thể thao) vào năm 1993. Giống bây giờ chủ yếu được nuôi như thú cưng ở Nhật Bản và các nước khác.
Một con Shina Inu đang chơi đùa ở bãi cỏ.
Tình trạng sức khỏe được biết ảnh hưởng đến giống chó này là dị ứng, thanh quang nhãn, cườm thủy tinh thể mắt, loạn sản xương hông, quặp và trật xương bánh chè. Nhìn chung, dù gì đi nữa, chúng có tính di truyền cao và khá nhiều Shiba được chẩn đoán khuyết tật do di truyền so với các giống chó khác.
Kiểm tra chung định kỳ được khuyến cáo nên được thực hiện trong suốt cuộc đời của con chó nhưng vấn đề thường được phát hiện sớm trong cuộc đời của nó. Kiểm tra mắt nên được thực hiện hàng năm vì vấn đề về mắt có thể phát triển theo thời gian. Năm hai tuổi, Shiba Inu có thể được coi là hoàn toàn tự do khỏi các vấn đề chung nếu không được phát hiện bởi thời điểm này, vì ở độ tuổi này bộ xương đã được phát triển đầy đủ.
Như đối với bất kỳ những con chó khác, Shiba nên được đi hoặc nếu không thì nên vận động hàng ngày.
Tuổi thọ trung bình của Shiba Inu là từ 12 đến 16 năm. Tập thể dục, đặc biệt là đi bộ mỗi ngày, sẽ giúp cho giống chó này sống lâu và khỏe mạnh. Shiba lâu đời nhất được biết đến là "Pusuke", đã qua đời ở tuổi 26 vào đầu tháng 12 năm 2011 và là chú chó già nhất còn sống vào thời điểm đó.
Giống chó này rất sạch sẽ, vì vậy nhu cầu chải chuốt nên được thực hiện tối thiểu. Một lớp lông Shiba Inu thô, ngắn có chiều dài trung bình với lớp lông bên ngoài dài 2,5 đến 3,2 cm (1 đến 1 1⁄4 in); và không thấm nước tự nhiên nên ít cần tắm thường xuyên. Nó cũng có một lớp lông dày có thể bảo vệ chúng khỏi nhiệt độ đông đá. Tuy nhiên, rụng lông có thể là một mối phiền toái. Rụng lông nặng nhất có sự thay đổi theo mùa và đặc biệt là trong mùa hè, nhưng việc chải lông hàng ngày có thể làm giảm vấn đề này. Chủ nhân không được phép cạo hoặc cắt lông của Shiba Inu, vì lông cần thiết để bảo vệ chó khỏi nhiệt độ cả nóng lẫn lạnh.
'''

In [110]:
#default model with number of senteces = 10
model = Summarizer()
result = model(body, num_sentences = 10)
print(result)

Shiba Inu (柴犬 (sài khuyển)) là loại chó nhỏ nhất trong sáu giống chó nguyên thủy và riêng biệt đến từ Nhật Bản. Điều này khiến cho một số người tin rằng Shiba được đặt tên như thế là vì loài chó này được sử dụng để săn mồi trong các bụi cây, hoặc có thể là do màu sắc phổ biến nhất của Shiba Inu là màu đỏ tương tự như của các cây bụi. Đối với con cái là 33 đến 41 cm (13 đến 16 in). Shiba có thể có màu đỏ, đen và nâu, hoặc màu vừng (màu đỏ với những sợi ngã sang đen), với một lớp lông lót màu kem, màu da bò, hoặc màu xám. Urajiro (màu kem trắng) có ở các bộ phận sau trên tất cả các vùng lông: ở hai bên mõm, trên má, bên trong tai, trên hàm dưới và ở chỗ cổ họng, bên trong chân, trên bụng, xung quanh các lỗ thông hơi và phía vùng bụng của đuôi. Shiba có tính chất tự lập và có thể dè dặt đối với người lạ nhưng lại trung thành và tình cảm với những người có được sự tôn trọng của nó. Nó có thể hung dữ với những con chó khác. Khi nghiên cứu về chó Nhật được chính thức hóa trong đầu và giữa th

In [107]:
#BERT large Uncased model with number of senteces = 10
model = Summarizer(model = 'bert-large-uncased')
result = model(body, num_sentences = 10)
print(result)

Shiba Inu (柴犬 (sài khuyển)) là loại chó nhỏ nhất trong sáu giống chó nguyên thủy và riêng biệt đến từ Nhật Bản. Tuy nhiên, trong một phương ngữ Nagano cổ, từ Shiba cũng có ý nghĩa là nhỏ, do đó cái tên có thể nói đến tầm vóc nhỏ bé của con chó. Khung hình của Shiba nhỏ gọn với cơ bắp phát triển tốt. Một tinh thần mạnh dạn, một bản chất tốt đẹp và sự thẳng thắn không lẫn lộn mang lại phẩm giá và vẻ đẹp tự nhiên. Nó có thể hung dữ với những con chó khác. Nó thường liếm bàn chân giống như mèo, thường di chuyển theo cách riêng của mình để giữ bộ lông sạch sẽ, nhưng lại cực kỳ thích bơi lội và chơi đùa trong các vũng nước. Vì bản chất khó tính và đầy kiêu hãnh vốn có, Shiba con rất dễ dạy dỗ và trong nhiều trường hợp sẽ tự dạy dỗ chính mình. Giống bây giờ chủ yếu được nuôi như thú cưng ở Nhật Bản và các nước khác. Tập thể dục, đặc biệt là đi bộ mỗi ngày, sẽ giúp cho giống chó này sống lâu và khỏe mạnh. Giống chó này rất sạch sẽ, vì vậy nhu cầu chải chuốt nên được thực hiện tối thiểu. Nó cũn

In [108]:
#BERT large Uncased model with ratio = 0.3
model = Summarizer(model = 'bert-large-uncased')
result = model(body, ratio = 0.3)
print(result)

Shiba Inu (柴犬 (sài khuyển)) là loại chó nhỏ nhất trong sáu giống chó nguyên thủy và riêng biệt đến từ Nhật Bản. Inu hoặc ken (犬 - Hán Việt: khuyển) trong tiếng Nhật có nghĩa là con chó, nhưng nguồn gốc của từ "Shiba" vẫn chưa rõ. Khung hình của Shiba nhỏ gọn với cơ bắp phát triển tốt. Đối với con cái là 33 đến 41 cm (13 đến 16 in). Giống chó cũng tương tác khá tốt với mèo. Nó có thể hung dữ với những con chó khác. Nó thường liếm bàn chân giống như mèo, thường di chuyển theo cách riêng của mình để giữ bộ lông sạch sẽ, nhưng lại cực kỳ thích bơi lội và chơi đùa trong các vũng nước. Vì bản chất khó tính và đầy kiêu hãnh vốn có, Shiba con rất dễ dạy dỗ và trong nhiều trường hợp sẽ tự dạy dỗ chính mình. Một đặc điểm giúp phân biệt giống chó này là "Shiba scream". Những dòng máu đó là Shinshu Shiba từ Nagano, Mino Shiba từ Gifu, và San'in Shiba từ Tottori và Shimane. San'in Shiba thì lớn hơn so với hầu hết các giống Shiba hiện nay, và thường có màu đen, không có dấu sẫm và trắng thường được 

In [109]:
#default model with CoreferenceHandler
model = Summarizer(model = 'bert-large-uncased', sentence_handler=CoreferenceHandler())
result = model(body, ratio = 0.3)
print(result)

Shiba Inu (柴犬 (sài khuyển)) là loại chó nhỏ nhất trong sáu giống chó nguyên thủy và riêng biệt đến từ Nhật Bản. Chúng là một giống chó nhỏ, nhanh nhẹn và thích hợp với địa hình miền núi, Shiba Inu ban đầu được nuôi để săn bắt. Inu hoặc ken (犬 - Hán Việt: khuyển) trong tiếng Nhật có nghĩa là con chó, nhưng nguồn gốc của từ "Shiba" vẫn chưa rõ. Khung hình của Shiba nhỏ gọn với cơ bắp phát triển tốt. Con đực có chiều cao từ 35 đến 43 cm (14 đến 17 in). Trọng lượng trung bình ở kích thước tương thích là khoảng 10 kg (22 lb) đối với con đực và 8 kg (18 lb) đối với con cái. Giống chó cũng tương tác khá tốt với mèo. Nó có thể hung dữ với những con chó khác. Nó thường liếm bàn chân giống như mèo, thường di chuyển theo cách riêng của mình để giữ bộ lông sạch sẽ, nhưng lại cực kỳ thích bơi lội và chơi đùa trong các vũng nước. Một đặc điểm giúp phân biệt giống chó này là "Shiba scream". Khi đủ kích động hay không vui, nó sẽ phát ra một tiếng thét lớn và cao. Năm 1954, một gia đình phục vụ vũ tran

In [111]:
model = Summarizer(model = 'xlnet-base-cased', sentence_handler=CoreferenceHandler())
result = model(body, ratio = 0.3)
print(result)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\transformers\configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


Shiba Inu (柴犬 (sài khuyển)) là loại chó nhỏ nhất trong sáu giống chó nguyên thủy và riêng biệt đến từ Nhật Bản. Từ Shiba (柴 - Hán Việt: sài) có nghĩa là cây bụi trong tiếng Nhật, đề cập đến một loại cây hoặc cây bụi có lá chuyển sang màu đỏ vào mùa thu. Điều này khiến cho một số người tin rằng Shiba được đặt tên như thế là vì loài chó này được sử dụng để săn mồi trong các bụi cây, hoặc có thể là do màu sắc phổ biến nhất của Shiba Inu là màu đỏ tương tự như của các cây bụi. Khung hình của Shiba nhỏ gọn với cơ bắp phát triển tốt. Trọng lượng trung bình ở kích thước tương thích là khoảng 10 kg (22 lb) đối với con đực và 8 kg (18 lb) đối với con cái. Lớp lông: Có hai lớp lông với lớp ngoài cứng và thẳng cùng một lớp trong mềm mại và dày. Màu đỏ: thương ở trên cổ họng, chóp ngực và ngực. Shiba có xu hướng thể hiện tính tự lập và đôi khi còn hung hăng. Shiba Inu tốt nhất nên được nuôi trong một gia đình mà không có những con chó nhỏ khác hay trẻ em, nhưng huấn luyện vâng lời vẫn có thể được 

In [113]:
result = model.run_embeddings(body, ratio=0.3)  # Will return (num_sentences +1, N) embedding numpy matrix.
result

array([[ 0.06264511, -0.04616188,  0.30910435, ..., -0.11081592,
        -0.06871919,  0.11080622],
       [ 0.0010872 , -0.0021876 ,  0.04671476, ...,  0.42539126,
        -0.261949  ,  0.09410372],
       [-0.03221155, -0.21116859,  0.19516714, ..., -0.12356774,
        -0.13606201,  0.36228496],
       ...,
       [-0.4348117 ,  0.3021262 ,  0.21845956, ..., -0.6307475 ,
        -0.06835769, -0.02482057],
       [-0.21573763,  0.32624435, -0.06969851, ..., -0.20903395,
        -0.34038976, -0.0154977 ],
       [-0.18740307, -0.29025468, -0.01592006, ..., -0.175454  ,
         0.24482788,  0.2545929 ]], dtype=float32)

In [114]:
res = model.calculate_elbow(body, k_max=10)
res

[2692.918701171875,
 2500.3896484375,
 2321.54345703125,
 2202.852783203125,
 2082.740966796875,
 2033.31689453125,
 1945.07568359375,
 1863.1490478515625,
 1775.525390625]

In [115]:
res = model.calculate_optimal_k(body, k_max=10)
res

3